In [ ]:
from sympy import *
init_printing()

In [ ]:
# state
x, y, z, vx, vy, vz = symbols('x y z v_x v_y v_z')
p                      = Matrix([x, y, z])
v                      = Matrix([vx, vy, vz])
s                      = Matrix([p, v])

In [ ]:
# costate
lx, ly, lz, lvx, lvy, lvz = symbols('\\lambda_x \\lambda_y \\lambda_z \\lambda_{v_x} \\lambda_{v_y} \\lambda_{v_z}')
lp                            = Matrix([lx, ly, lz])
lv                            = Matrix([lvx, lvy, lvz])
l                             = Matrix([lp, lv])

In [ ]:
# fullstate
fs = Matrix([s, l])
fs

In [ ]:
# control: throttle and direction
ut, ub, ux, uy, uz = symbols("u_t u_b u_x, u_y u_z")
i = Matrix([ux, uy, uz])
actions = Matrix([ut, ub, ux, uy, uz])

In [ ]:
# physical parametres
mass, gravity, density, CD, planaform, volume, thrust = symbols("m g \\rho C_D A V T_{max}")
vfx, vfy, vfz = symbols("v_{f_x} v_{f_y} v_{f_z}")
vf = Matrix([vfx, vfy, vfz])

# optimisation parametres
alpha = symbols("\\alpha")

In [ ]:
# Lagrange multiplier
gamma, beta = symbols("\\gamma \\beta")

In [ ]:
# gravity
g = Matrix([0, 0, -gravity])

# velocity relative to stream
vinf = v - vf

# velocity magnitude
vmag = sqrt(vinf[0]**2 + vinf[1]**2 + vinf[2]**2)

# velocity direction
vhat = v/vmag

# gravity
Fg = mass*g

# fluid drag
Fd = - Rational(1,2)*density*vmag**2*vhat*CD*planaform

# buoyancy
Fb = -density*volume*g*ub

# thrust
Ft = thrust*ut*i

In [ ]:
# rate of change of position
dr = v

# rate of change of velocity
dv = (Fg + Fd + Fb + Ft)/mass

# state equations of motion
ds = Matrix([dr, dv])
ds

In [ ]:
# Lagrangian
L = ut
L = alpha*L + (1 - alpha)*L**2
L

In [ ]:
# Hamiltonian
H = l.dot(ds) + L
H

In [ ]:
# terms of Hamiltonian dependant on thrust direction
Hi = sum([term for term in expand(H).args if (ux in term.free_symbols or uy in term.free_symbols or uz in term.free_symbols)])
dell = [diff(Hi + gamma*(sqrt(ux**2 + uy**2 + uz**2) - 1), var) for var in [ux, uy, uz, gamma]]
uxstar, uystar, uzstar, gammastar = solve(dell, [ux, uy, uz, gamma])[0]
istar = Matrix([uxstar, uystar, uzstar])
istar

In [ ]:
H = H.subs(ux, uxstar).subs(uy, uystar).subs(uz, uzstar)
H

In [ ]:
# terms of Hamiltonian dependant on thrust magnitude
Hut = sum([term for term in expand(H).args if (ut in term.free_symbols)])
# Lagrangian
dell = [diff(Hut, var) for var in [ut]]
utstar = solve(dell, [ut])[ut]
utstar

In [ ]:
H = H.subs(ut, utstar)
H

In [ ]:
Hub = sum([term for term in expand(H).args if (ub in term.free_symbols)])
dell = [diff(H, var) for var in [ub]]
Hub